In [3]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.layers.activation import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras import optimizers
import keras.backend as K

In [4]:
# Create the dataframe for the dataset
df = pd.read_csv('CBC_data.csv')
# Calculating the mean of every column (feature)
rbc_mean = df['RBC'].mean()
hgb_mean = df['HGB'].mean()
wbc_mean = df['WBC'].mean()
eos_cnt_mean = df['EOS.CNT'].mean()
mono_cnt_mean = df['MONO.CNT'].mean()
nut_cnt_mean = df['NUT.CNT'].mean()
pl_cnt_mean = df['PL.CNT'].mean()
lymp_cnt_mean = df['LYMP.CNT'].mean()
# Filling the all the missing values 
df['RBC'].fillna(rbc_mean, inplace=True)
df['HGB'].fillna(hgb_mean, inplace=True)
df['WBC'].fillna(wbc_mean, inplace=True)
df['EOS.CNT'].fillna(eos_cnt_mean, inplace=True)
df['MONO.CNT'].fillna(mono_cnt_mean, inplace=True)
df['NUT.CNT'].fillna(nut_cnt_mean, inplace=True)
df['PL.CNT'].fillna(pl_cnt_mean, inplace=True)
df['LYMP.CNT'].fillna(lymp_cnt_mean, inplace=True)
# Converts the dataframe into a numpy array
data = df.to_numpy()
# Converts the features that are not numerical values into integer values
encoder = OrdinalEncoder()
data_to_encode = data[:,[1, 2,3,4,6]]
encoder.fit(data_to_encode)
data[:, [1, 2,3,4,6]] = encoder.transform(data_to_encode)
data = data[:,[1,2,4,5,6,7,8,9,10,11,12,13,14]]
X_train_data = data[:300]
X_test_data = data[300:420]
print(X_train_data[0])
# X_train_data = data[:300]
# X_test_data = data[300:420]

[1.0 1.0 0.0 9 0.0 6.4 16.6 14.2 142.0 852.0 6390.0 210.0 6816.0]


In [5]:
pos_examples_count, neg_examples_count  = 0, 0
train_idx, test_idx = 0, 0
pos_check_count=0
neg_check_count=0
for i in range(len(data)):
    pos_check_count +=1
    if pos_examples_count == 150:
        break
    if data[i][4] == 1.0:
        X_train_data[train_idx,:] = data[i,:]
        pos_examples_count += 1
        train_idx += 1

for i in range(len(data)):
    neg_check_count +=1
    if neg_examples_count == 150:
        break
    if data[i][4] == 0.0:
        X_train_data[train_idx,:] = data[i,:]
        neg_examples_count += 1
        train_idx += 1

pos_examples_count, neg_examples_count = 0, 0
for i in range(pos_check_count, len(data)):
    if pos_examples_count == 60:
        break
    if data[i][4] == 1.0:
        X_test_data[test_idx,:] = data[i,:]
        pos_examples_count += 1
        test_idx += 1

for i in range(neg_check_count, len(data)):
    if neg_examples_count == 60:
        break
    if data[i][4] == 0.0:
        X_test_data[test_idx,:] = data[i,:]
        neg_examples_count += 1
        test_idx += 1


y_train = X_train_data[:, 4:5]
X_train = X_train_data[:, 5:]
X_train = np.hstack((X_train, X_train_data[:, [0,1, 2, 3]]))
y_test = X_test_data[:, 4:5]
X_test = X_test_data[:, 5:]
X_test = np.hstack((X_test, X_test_data[:, [0,1, 2, 3]]))

In [6]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train).astype(np.float32)
X_test_std = scaler.transform(X_test).astype(np.float32)
y_train = np.ravel(y_train).astype(np.int32)
y_test = np.ravel(y_test).astype(np.int32)
print(X_train_std)

[[-0.80121654 -0.5129542   1.1968712  ...  0.4260578  -0.69652605
  -0.6533068 ]
 [-0.80121654 -0.5740137   0.25312668 ... -1.212626   -0.69652605
   1.881756  ]
 [-0.9643027  -0.6961328  -0.18244769 ...  0.4260578  -0.69652605
   1.7233145 ]
 ...
 [ 0.82964474  1.196713   -0.40023488 ...  2.0647416   1.4356965
   0.6142246 ]
 [-0.31195816  0.0365817   0.07163735 ... -1.212626   -0.69652605
  -0.6533068 ]
 [ 1.3189031   0.21976031  1.3420626  ...  0.4260578  -0.69652605
  -0.6533068 ]]


In [54]:
# three_hlayer= MLPClassifier(hidden_layer_sizes=(64,32,16,8), max_iter=1000, alpha = .001, learning_rate_init=.001)
one_hlayer= MLPClassifier(random_state=12, learning_rate_init=.0001, max_iter=1000)
# three_hlayer.fit(X_train_std, y_train)
# acc = three_hlayer.score(X_test_std, y_test)
# print("three",acc)
grid = {
    "hidden_layer_sizes":[(67), (66), (64), (100), (48)],
    "alpha":[(.001),(.01),(.1)],
}
grid_search = GridSearchCV(one_hlayer, grid, cv=5)
grid_search.fit(X_train, y_train)
print("acc", grid_search.best_params_)
print("acc", grid_search.best_score_)



acc {'alpha': 0.001, 'hidden_layer_sizes': 66}
acc 0.5366666666666666


In [28]:
three_hlayer= MLPClassifier(random_state=12, learning_rate_init=.0001, max_iter=1000, alpha = .1)

grid = {
    "hidden_layer_sizes":[(66,32,8),(64,32,8), (64,32,8)],
    "alpha":[(.001),(.01),(.1)],
}

grid_search = GridSearchCV(three_hlayer, grid, cv=5)
grid_search.fit(X_train, y_train)
print("acc", grid_search.best_params_)
print("acc", grid_search.best_score_)





acc {'alpha': 0.1, 'hidden_layer_sizes': (64, 32, 8)}
acc 0.54


In [8]:
# three_hlayer= MLPClassifier(hidden_layer_sizes=(64,32,16,8), max_iter=1000, alpha = .001, learning_rate_init=.001)
five_hlayer= MLPClassifier(random_state=12, learning_rate_init=.0001, max_iter=1000)
# three_hlayer.fit(X_train_std, y_train)
# acc = three_hlayer.score(X_test_std, y_test)
# print("three",acc)
grid = {
    "hidden_layer_sizes":[(64,32,16,8,4),(64,60,16,8,4), (72,32,16,8,4), (32,24,16,8,4)],
    "alpha":[(.001),(.01),(.1)],
}
grid_search = GridSearchCV(five_hlayer, grid, cv=5)
grid_search.fit(X_train, y_train)
print("acc", grid_search.best_params_)
print("acc", grid_search.best_score_)


acc {'alpha': 0.001, 'hidden_layer_sizes': (64, 32, 16, 8, 4)}
acc 0.55


In [18]:
np.random.seed(0)
tf.random.set_seed(0)
param_lst = [0.001, 0.1, 1, 10, 100]
types = ['tanh', 'relu', 'sigmoid']
batch = 32

In [19]:
def create_mod_1l(activ):
  model = Sequential()
  model.add(Dense(66, activation=activ, input_shape=(12,), kernel_initializer='glorot_uniform'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  return model
def create_mod_1l_with_l1reg(activ,param):
  model = Sequential()
  model.add(Dense(66, activation=activ, input_shape=(12,), kernel_regularizer = tf.keras.regularizers.l1( l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  return model
def create_mod_1l_with_l2reg(activ,param):
  model = Sequential()
  model.add(Dense(66, activation=activ, input_shape=(12,), kernel_regularizer = tf.keras.regularizers.l2( l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [22]:
training_acc_dict = {}
val_acc_dict = {}
for activate in types:
  model = create_mod_1l(activate)
  optimizer = optimizers.Adam()
  model.compile(loss= keras.losses.binary_crossentropy, optimizer=optimizer, metrics=['accuracy'])  
  history = model.fit(X_train_std, y_train, epochs=5, batch_size=batch, validation_data=(X_test_std, y_test))
  train_acc =history.history['accuracy'][-1]
  val_acc= history.history['val_accuracy'][-1]
  print(activate)
  print(train_acc)
  print(val_acc)
  if activate in training_acc_dict:
    training_acc_dict[activate]=max(train_acc, training_acc_dict[activate])
    val_acc_dict[activate]= max(val_acc, training_acc_dict[activate])
  else:
    val_acc_dict[activate]=val_acc
    training_acc_dict[activate]=train_acc

Epoch 1/5
10/10 [==============================] - 1s 25ms/step - loss: 0.8180 - accuracy: 0.4800 - val_loss: 0.7313 - val_accuracy: 0.5167
Epoch 2/5
10/10 [==============================] - 0s 6ms/step - loss: 0.7675 - accuracy: 0.5000 - val_loss: 0.7411 - val_accuracy: 0.5000
Epoch 3/5
10/10 [==============================] - 0s 8ms/step - loss: 0.6971 - accuracy: 0.5900 - val_loss: 0.7466 - val_accuracy: 0.4833
Epoch 4/5
10/10 [==============================] - 0s 8ms/step - loss: 0.7240 - accuracy: 0.5433 - val_loss: 0.7504 - val_accuracy: 0.4917
Epoch 5/5
10/10 [==============================] - 0s 7ms/step - loss: 0.7078 - accuracy: 0.5467 - val_loss: 0.7538 - val_accuracy: 0.5000
tanh
0.54666668176651
0.5
Epoch 1/5
10/10 [==============================] - 1s 32ms/step - loss: 0.7146 - accuracy: 0.5333 - val_loss: 0.7034 - val_accuracy: 0.4917
Epoch 2/5
10/10 [==============================] - 0s 8ms/step - loss: 0.7271 - accuracy: 0.5600 - val_loss: 0.7032 - val_accuracy: 0.4833

In [24]:
training_acc_dict = {}
val_acc_dict = {}
for activate in types:
  for reg_weight in param_lst:
    model = create_mod_1l_with_l1reg(activate, reg_weight)
    optimizer = optimizers.Adam()
    model.compile(loss= keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])  
    history = model.fit(X_train_std, y_train, epochs=5, batch_size=batch, validation_data=(X_test_std, y_test))
    train_acc =history.history['accuracy'][-1]
    val_acc= history.history['val_accuracy'][-1]
    print(activate)
    print(reg_weight)
    print(train_acc)
    print(val_acc)
    training_acc_dict[activate]=train_acc
    val_acc_dict[activate]=val_acc

Epoch 1/5
10/10 [==============================] - 1s 38ms/step - loss: 0.1075 - accuracy: 0.5567 - val_loss: 0.1070 - val_accuracy: 0.4333
Epoch 2/5
10/10 [==============================] - 0s 7ms/step - loss: 0.1067 - accuracy: 0.5800 - val_loss: 0.1063 - val_accuracy: 0.4333
Epoch 3/5
10/10 [==============================] - 0s 10ms/step - loss: 0.1060 - accuracy: 0.5633 - val_loss: 0.1056 - val_accuracy: 0.4500
Epoch 4/5
10/10 [==============================] - 0s 8ms/step - loss: 0.1054 - accuracy: 0.6067 - val_loss: 0.1051 - val_accuracy: 0.4417
Epoch 5/5
10/10 [==============================] - 0s 9ms/step - loss: 0.1049 - accuracy: 0.5867 - val_loss: 0.1046 - val_accuracy: 0.4333
tanh
0.001
0.5866666436195374
0.4333333373069763
Epoch 1/5
10/10 [==============================] - 1s 28ms/step - loss: 10.3074 - accuracy: 0.5600 - val_loss: 9.8746 - val_accuracy: 0.4917
Epoch 2/5
10/10 [==============================] - 0s 5ms/step - loss: 9.5728 - accuracy: 0.5433 - val_loss: 9.16

In [25]:
training_acc_dict = {}
val_acc_dict = {}
for activate in types:
  for reg_weight in param_lst:
    model = create_mod_1l_with_l2reg(activate, reg_weight)
    optimizer = optimizers.Adam()
    model.compile(loss= keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])  
    history = model.fit(X_train_std, y_train, epochs=5, batch_size=batch, validation_data=(X_test_std, y_test))
    train_acc =history.history['accuracy'][-1]
    val_acc= history.history['val_accuracy'][-1]
    print(activate)
    print(reg_weight)
    print(train_acc)
    print(val_acc)
    training_acc_dict[activate]=train_acc
    val_acc_dict[activate]=val_acc

Epoch 1/5
10/10 [==============================] - 1s 43ms/step - loss: 0.0205 - accuracy: 0.4867 - val_loss: 0.0204 - val_accuracy: 0.4917
Epoch 2/5
10/10 [==============================] - 0s 12ms/step - loss: 0.0203 - accuracy: 0.4633 - val_loss: 0.0202 - val_accuracy: 0.4917
Epoch 3/5
10/10 [==============================] - 0s 11ms/step - loss: 0.0201 - accuracy: 0.5167 - val_loss: 0.0200 - val_accuracy: 0.4417
Epoch 4/5
10/10 [==============================] - 0s 10ms/step - loss: 0.0200 - accuracy: 0.5200 - val_loss: 0.0199 - val_accuracy: 0.4667
Epoch 5/5
10/10 [==============================] - 0s 9ms/step - loss: 0.0198 - accuracy: 0.5367 - val_loss: 0.0198 - val_accuracy: 0.5250
tanh
0.001
0.5366666913032532
0.5249999761581421
Epoch 1/5
10/10 [==============================] - 1s 34ms/step - loss: 1.9025 - accuracy: 0.5567 - val_loss: 1.8027 - val_accuracy: 0.5167
Epoch 2/5
10/10 [==============================] - 0s 7ms/step - loss: 1.7356 - accuracy: 0.5533 - val_loss: 1.6

In [26]:
def create_mod_3l(activ):
  model = Sequential()
  model.add(Dense(64, activation=activ, input_shape=(12,), kernel_initializer='glorot_uniform'))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation=activ))
  model.add(Dropout(0.5))
  model.add(Dense(8, activation=activ))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  return model
def create_mod_3l_with_l1reg(activ,param):
  model = Sequential()
  model.add(Dense(64, activation=activ, input_shape=(12,), kernel_regularizer = tf.keras.regularizers.l1( l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(8, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  return model
def create_mod_3l_with_l2reg(activ,param):
  model = Sequential()
  model.add(Dense(64, activation=activ, input_shape=(12,), kernel_regularizer = tf.keras.regularizers.l2( l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(8, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [27]:
training_acc_dict = {}
val_acc_dict = {}
for activate in types:
  model = create_mod_3l(activate)
  optimizer = optimizers.Adam()
  model.compile(loss= keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])  
  history = model.fit(X_train_std, y_train, epochs=5, batch_size=batch, validation_data=(X_test_std, y_test))
  train_acc =history.history['accuracy'][-1]
  val_acc= history.history['val_accuracy'][-1]
  print(activate)
  print(train_acc)
  print(val_acc)
  training_acc_dict[activate]=train_acc, 
  val_acc_dict[activate]=val_acc

Epoch 1/5
10/10 [==============================] - 1s 30ms/step - loss: 0.0000e+00 - accuracy: 0.5367 - val_loss: 0.0000e+00 - val_accuracy: 0.5583
Epoch 2/5
10/10 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 0.5300 - val_loss: 0.0000e+00 - val_accuracy: 0.5583
Epoch 3/5
10/10 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 0.5067 - val_loss: 0.0000e+00 - val_accuracy: 0.5500
Epoch 4/5
10/10 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 0.5400 - val_loss: 0.0000e+00 - val_accuracy: 0.5333
Epoch 5/5
10/10 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 0.5033 - val_loss: 0.0000e+00 - val_accuracy: 0.5417
tanh
0.503333330154419
0.5416666865348816
Epoch 1/5
10/10 [==============================] - 1s 43ms/step - loss: 0.0000e+00 - accuracy: 0.5033 - val_loss: 0.0000e+00 - val_accuracy: 0.4583
Epoch 2/5
10/10 [==============================] - 0s 8ms/step - loss: 0

In [31]:
training_acc_dict = {}
val_acc_dict = {}
for activate in types:
  for reg_weight in param_lst:
    model = create_mod_3l_with_l1reg(activate, reg_weight)
    optimizer = optimizers.Adam()
    model.compile(loss= keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])  
    history = model.fit(X_train_std, y_train, epochs=5, batch_size=batch, validation_data=(X_test_std, y_test))
    train_acc =history.history['accuracy'][-1]
    val_acc= history.history['val_accuracy'][-1]
    print(activate)
    print(reg_weight)
    print(train_acc)
    print(val_acc)
    training_acc_dict[activate]=train_acc
    val_acc_dict[activate]=val_acc

Epoch 1/5
10/10 [==============================] - 1s 33ms/step - loss: 0.1613 - accuracy: 0.4933 - val_loss: 0.1611 - val_accuracy: 0.5667
Epoch 2/5
10/10 [==============================] - 0s 9ms/step - loss: 0.1610 - accuracy: 0.5133 - val_loss: 0.1609 - val_accuracy: 0.5500
Epoch 3/5
10/10 [==============================] - 0s 10ms/step - loss: 0.1607 - accuracy: 0.5167 - val_loss: 0.1605 - val_accuracy: 0.5250
Epoch 4/5
10/10 [==============================] - 0s 8ms/step - loss: 0.1603 - accuracy: 0.5133 - val_loss: 0.1600 - val_accuracy: 0.5500
Epoch 5/5
10/10 [==============================] - 0s 10ms/step - loss: 0.1598 - accuracy: 0.5700 - val_loss: 0.1596 - val_accuracy: 0.5333
tanh
0.001
0.5699999928474426
0.5333333611488342
Epoch 1/5
10/10 [==============================] - 1s 28ms/step - loss: 15.6268 - accuracy: 0.5167 - val_loss: 14.9554 - val_accuracy: 0.4833
Epoch 2/5
10/10 [==============================] - 0s 8ms/step - loss: 14.4925 - accuracy: 0.5233 - val_loss: 1

In [30]:
training_acc_dict = {}
val_acc_dict = {}
for activate in types:
  for reg_weight in param_lst:
    model = create_mod_3l_with_l2reg(activate, reg_weight)
    optimizer = optimizers.Adam()
    model.compile(loss= keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])  
    history = model.fit(X_train_std, y_train, epochs=5, batch_size=batch, validation_data=(X_test_std, y_test))
    train_acc =history.history['accuracy'][-1]
    val_acc= history.history['val_accuracy'][-1]
    print(activate)
    print(reg_weight)
    print(train_acc)
    print(val_acc)
    training_acc_dict[activate]=train_acc
    val_acc_dict[activate]=val_acc

Epoch 1/5
10/10 [==============================] - 1s 33ms/step - loss: 0.0724 - accuracy: 0.5000 - val_loss: 0.0724 - val_accuracy: 0.5250
Epoch 2/5
10/10 [==============================] - 0s 9ms/step - loss: 0.0724 - accuracy: 0.5200 - val_loss: 0.0724 - val_accuracy: 0.4750
Epoch 3/5
10/10 [==============================] - 0s 7ms/step - loss: 0.0723 - accuracy: 0.5467 - val_loss: 0.0723 - val_accuracy: 0.4750
Epoch 4/5
10/10 [==============================] - 0s 7ms/step - loss: 0.0723 - accuracy: 0.5267 - val_loss: 0.0723 - val_accuracy: 0.4667
Epoch 5/5
10/10 [==============================] - 0s 7ms/step - loss: 0.0723 - accuracy: 0.5600 - val_loss: 0.0722 - val_accuracy: 0.4667
tanh
0.001
0.5600000023841858
0.46666666865348816
Epoch 1/5
10/10 [==============================] - 2s 30ms/step - loss: 7.3127 - accuracy: 0.4433 - val_loss: 6.9833 - val_accuracy: 0.5000
Epoch 2/5
10/10 [==============================] - 0s 10ms/step - loss: 6.7569 - accuracy: 0.4833 - val_loss: 6.45

In [33]:
def create_mod_5l(activ):
  model = Sequential()
  model.add(Dense(64, activation=activ, input_shape=(12,), kernel_initializer='glorot_uniform'))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation=activ))
  model.add(Dropout(0.5))
  model.add(Dense(16, activation=activ))
  model.add(Dropout(0.5))
  model.add(Dense(8, activation=activ))
  model.add(Dropout(0.5))
  model.add(Dense(4, activation=activ))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  return model
def create_mod_5l_with_l1reg(activ,param):
  model = Sequential()
  model.add(Dense(64, activation=activ, input_shape=(12,), kernel_regularizer = tf.keras.regularizers.l1( l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(16, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(8, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(4, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  return model
def create_mod_5l_with_l2reg(activ,param):
  model = Sequential()
  model.add(Dense(64, activation=activ, input_shape=(12,), kernel_regularizer = tf.keras.regularizers.l2( l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(16, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(8, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(4, activation=activ, kernel_regularizer = tf.keras.regularizers.l2(l=param)))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [34]:
training_acc_dict = {}
val_acc_dict = {}
for activate in types:
  model = create_mod_5l(activate)
  optimizer = optimizers.Adam()
  model.compile(loss= keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])  
  history = model.fit(X_train_std, y_train, epochs=5, batch_size=batch, validation_data=(X_test_std, y_test))
  train_acc =history.history['accuracy'][-1]
  val_acc= history.history['val_accuracy'][-1]
  print(activate)
  print(train_acc)
  print(val_acc)
  training_acc_dict[activate]=train_acc
  val_acc_dict[activate]=val_acc

Epoch 1/5
10/10 [==============================] - 1s 37ms/step - loss: 0.0000e+00 - accuracy: 0.5267 - val_loss: 0.0000e+00 - val_accuracy: 0.5250
Epoch 2/5
10/10 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.4967 - val_loss: 0.0000e+00 - val_accuracy: 0.5500
Epoch 3/5
10/10 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 0.5200 - val_loss: 0.0000e+00 - val_accuracy: 0.5250
Epoch 4/5
10/10 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 0.5067 - val_loss: 0.0000e+00 - val_accuracy: 0.4833
Epoch 5/5
10/10 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 0.4533 - val_loss: 0.0000e+00 - val_accuracy: 0.4750
tanh
0.4533333480358124
0.4749999940395355
Epoch 1/5
10/10 [==============================] - 1s 34ms/step - loss: 0.0000e+00 - accuracy: 0.5200 - val_loss: 0.0000e+00 - val_accuracy: 0.5167
Epoch 2/5
10/10 [==============================] - 0s 10ms/step - loss: 0

In [35]:
training_acc_dict = {}
val_acc_dict = {}
for activate in types:
  for reg_weight in param_lst:
    model = create_mod_5l_with_l1reg(activate, reg_weight)
    optimizer = optimizers.Adam()
    model.compile(loss= keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])  
    history = model.fit(X_train_std, y_train, epochs=5, batch_size=batch, validation_data=(X_test_std, y_test))
    train_acc =history.history['accuracy'][-1]
    val_acc= history.history['val_accuracy'][-1]
    print(activate)
    print(reg_weight)
    print(train_acc)
    print(val_acc)
    training_acc_dict[activate]=train_acc
    val_acc_dict[activate]=val_acc

Epoch 1/5
10/10 [==============================] - 1s 33ms/step - loss: 0.1917 - accuracy: 0.4600 - val_loss: 0.1915 - val_accuracy: 0.5167
Epoch 2/5
10/10 [==============================] - 0s 8ms/step - loss: 0.1914 - accuracy: 0.5400 - val_loss: 0.1912 - val_accuracy: 0.5167
Epoch 3/5
10/10 [==============================] - 0s 8ms/step - loss: 0.1911 - accuracy: 0.5167 - val_loss: 0.1909 - val_accuracy: 0.5167
Epoch 4/5
10/10 [==============================] - 0s 9ms/step - loss: 0.1908 - accuracy: 0.4767 - val_loss: 0.1906 - val_accuracy: 0.5083
Epoch 5/5
10/10 [==============================] - 0s 8ms/step - loss: 0.1905 - accuracy: 0.4933 - val_loss: 0.1903 - val_accuracy: 0.4917
tanh
0.001
0.4933333396911621
0.49166667461395264
Epoch 1/5
10/10 [==============================] - 1s 37ms/step - loss: 18.4315 - accuracy: 0.4733 - val_loss: 17.7101 - val_accuracy: 0.3667
Epoch 2/5
10/10 [==============================] - 0s 9ms/step - loss: 17.2243 - accuracy: 0.4400 - val_loss: 16

In [36]:
training_acc_dict = {}
val_acc_dict = {}
for activate in types:
  for reg_weight in param_lst:
    model = create_mod_5l_with_l2reg(activate, reg_weight)
    optimizer = optimizers.Adam()
    model.compile(loss= keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])  
    history = model.fit(X_train_std, y_train, epochs=5, batch_size=batch, validation_data=(X_test_std, y_test))
    train_acc =history.history['accuracy'][-1]
    val_acc= history.history['val_accuracy'][-1]
    print(activate)
    print(reg_weight)
    print(train_acc)
    print(val_acc)
    training_acc_dict[activate]=train_acc
    val_acc_dict[activate]=val_acc

Epoch 1/30
10/10 [==============================] - 1s 31ms/step - loss: 0.0985 - accuracy: 0.5000 - val_loss: 0.0985 - val_accuracy: 0.5500
Epoch 2/30
10/10 [==============================] - 0s 6ms/step - loss: 0.0985 - accuracy: 0.5033 - val_loss: 0.0984 - val_accuracy: 0.5583
Epoch 3/30
10/10 [==============================] - 0s 6ms/step - loss: 0.0983 - accuracy: 0.5233 - val_loss: 0.0983 - val_accuracy: 0.5333
Epoch 4/30
10/10 [==============================] - 0s 6ms/step - loss: 0.0982 - accuracy: 0.5167 - val_loss: 0.0982 - val_accuracy: 0.5250
Epoch 5/30
10/10 [==============================] - 0s 7ms/step - loss: 0.0981 - accuracy: 0.4600 - val_loss: 0.0980 - val_accuracy: 0.5083
Epoch 6/30
10/10 [==============================] - 0s 8ms/step - loss: 0.0979 - accuracy: 0.5067 - val_loss: 0.0978 - val_accuracy: 0.5250
Epoch 7/30
10/10 [==============================] - 0s 8ms/step - loss: 0.0977 - accuracy: 0.5200 - val_loss: 0.0976 - val_accuracy: 0.5167
Epoch 8/30
10/10 [=